__В блокноте text_classification.ipynb разобрали анализ тональности используя полносвязанную сеть сделать на тех же данных__

__1. tf-idf/count vectorizer + логистическую регрессию__

__2. Обучить вашу архитектуре сети возможно туже что и была на занятии проверить:__

__•	взять предобученный эмбединг(к примеру word2vec) и загрузить в слой Embedding__

__•	взять слой Embedding без предобученных весов__

__Сравнить все подходы в том числе и полносвязанную сеть что лучше отработало__

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd

from tqdm import tqdm
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model, metrics

from tensorflow.keras import layers, Model, Sequential, losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, \
    Flatten, GlobalAveragePooling1D, Reshape, Dropout

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 2s 0us/step


In [3]:
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [4]:
def get_df_from_files(directory, labels_dirs):
    
    texts = []
    labels = []

    for i, tag in enumerate(labels_dirs):
        path_tag = os.path.join(directory, tag)
        
        for name in tqdm(os.listdir(path_tag)):
            file = os.path.join(path_tag, name)
            with open(file, 'r', encoding='utf-8') as t:
                texts.append(t.read())
                labels.append(i)

    return pd.DataFrame({'text': texts, 'label': labels}).sample(frac=1).reset_index(drop=True) 

In [5]:
# neg=1, pos=0
labels_dirs = ['neg', 'pos']
train = get_df_from_files(train_dir, labels_dirs)
test = get_df_from_files(test_dir, labels_dirs)

100%|██████████| 12500/12500 [00:00<00:00, 37014.67it/s]


In [6]:
train.head()

,text,label
0,I still find it difficult to comprehend that a...,0
1,"To be as honest as I possibly can, The Devil's...",0
2,An old family story told to two young girls by...,1
3,Ulises is a literature teacher that arrives to...,0
4,I caught this Cuban film at at an arthouse fil...,0


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
def removeApostrophe(review):
    phrase = re.sub(r"won't", "will not", review)
    phrase = re.sub(r"can\'t", "can not", review)
    phrase = re.sub(r"n\'t", " not", review)
    phrase = re.sub(r"\'re", " are", review)
    phrase = re.sub(r"\'s", " is", review)
    phrase = re.sub(r"\'d", " would", review)
    phrase = re.sub(r"\'ll", " will", review)
    phrase = re.sub(r"\'t", " not", review)
    phrase = re.sub(r"\'ve", " have", review)
    phrase = re.sub(r"\'m", " am", review)
    return phrase

def removeApostropheFixed(review):
    phrase = re.sub(r"won't", "willnot", review)
    phrase = re.sub(r"can\'t", "cannot", review)
    phrase = re.sub(r"\snot", "not", review)
    phrase = re.sub(r"n\'t", "not", review)
    phrase = re.sub(r"\'re", " are", review)
    phrase = re.sub(r"\'s", " is", review)
    phrase = re.sub(r"\'d", " would", review)
    phrase = re.sub(r"\'ll", " will", review)
    phrase = re.sub(r"\'t", " not", review)
    phrase = re.sub(r"\'ve", " have", review)
    phrase = re.sub(r"\'m", " am", review)
    return phrase

def removeSpecialChars(review):
     return re.sub('[^a-zA-Z]', ' ', review)
    
def doCleaningFixed(review):
    review = removeApostropheFixed(review)
    review = removeSpecialChars(review) 
    review = review.lower()  
    # review = review.split() #Tokenization
    # lmtzr = WordNetLemmatizer()
    # review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    # review = " ".join(review)    
    return review

def doTextCleaning(review):
    review = removeApostrophe(review)
    review = removeSpecialChars(review) 
    review = review.lower()  
    # review = review.split() #Tokenization
    # lmtzr = WordNetLemmatizer()
    # review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    # review = " ".join(review)    
    return review

In [9]:
train['clean_text'] = train['text'].apply(doTextCleaning)
test['clean_text'] = test['text'].apply(doTextCleaning)

In [10]:
train

,text,label,clean_text
0,I still find it difficult to comprehend that a...,0,i still find it difficult to comprehend that a...
1,"To be as honest as I possibly can, The Devil's...",0,to be as honest as i possibly can the devil s...
2,An old family story told to two young girls by...,1,an old family story told to two young girls by...
3,Ulises is a literature teacher that arrives to...,0,ulises is a literature teacher that arrives to...
4,I caught this Cuban film at at an arthouse fil...,0,i caught this cuban film at at an arthouse fil...
...,...,...,...
24995,The most agile fat guy in martial arts does it...,1,the most agile fat guy in martial arts does it...
24996,The first thing you meet when you study fascis...,1,the first thing you meet when you study fascis...
24997,This is a wonderful film. The non-stop patter ...,1,this is a wonderful film the non stop patter ...
24998,"We've all been there, sitting with some friend...",0,we ve all been there sitting with some friend...


In [11]:
train_x, train_y = train['clean_text'], train['label']
test_x, test_y = test['clean_text'], test['label']

In [12]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train['clean_text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xtest_tfidf =  tfidf_vect.transform(test_x)

In [13]:
lr_model = linear_model.LogisticRegression(random_state=27, max_iter=300, n_jobs=-1)

lr_model.fit(xtrain_tfidf, train_y)
accuracy_lr = metrics.accuracy_score(lr_model.predict(xtest_tfidf), test_y)

In [14]:
batch_size = 64
seed = 27

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [15]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [16]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [17]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [18]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [19]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [20]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
embedding_dim = 64

In [22]:
model = tf.keras.Sequential([
    Embedding(max_features + 1, embedding_dim),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(145),
    Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 145)               9425      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 146       
Total params: 649,635
Trainable params: 649,635
Non-trainable params: 0
__________________________________________________

In [23]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [24]:
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/20
313/313 [==============================] - 14s 45ms/step - loss: 0.4859 - binary_accuracy: 0.7578 - val_loss: 0.3076 - val_binary_accuracy: 0.8762
Epoch 2/20
313/313 [==============================] - 8s 24ms/step - loss: 0.2580 - binary_accuracy: 0.8975 - val_loss: 0.2848 - val_binary_accuracy: 0.8880
Epoch 3/20
313/313 [==============================] - 8s 24ms/step - loss: 0.1961 - binary_accuracy: 0.9251 - val_loss: 0.3034 - val_binary_accuracy: 0.8830
Epoch 4/20
313/313 [==============================] - 8s 25ms/step - loss: 0.1570 - binary_accuracy: 0.9431 - val_loss: 0.3392 - val_binary_accuracy: 0.8810
Epoch 5/20
313/313 [==============================] - 8s 24ms/step - loss: 0.1277 - binary_accuracy: 0.9553 - val_loss: 0.3905 - val_binary_accuracy: 0.8750
Epoch 6/20
313/313 [==============================] - 8s 24ms/step - loss: 0.1034 - binary_accuracy: 0.9658 - val_loss: 0.4620 - val_binary_accuracy: 0.8694
Epoch 7/20
313/313 [==============================] - 8s 

In [25]:
loss, accuracy_net = model.evaluate(test_ds)

391/391 [==============================] - 10s 25ms/step - loss: 1.3863 - binary_accuracy: 0.8278


In [26]:
print(f"Accuracy Logistic Regression = {accuracy_lr}")
print(f"Accuracy NET = {accuracy_net}")

Accuracy Logistic Regression = 0.88272
Accuracy NET = 0.8278399705886841
